In [16]:
!pip install cleverhans

import torch
import torchvision
from torchvision import datasets, transforms, models
from cleverhans.torch.attacks.projected_gradient_descent import projected_gradient_descent
import matplotlib.pyplot as plt
import numpy as np
import os

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
import os
os.chdir('C:\\Users\\Admin\\Desktop\\Python\\corruption-testing')

from experiments import network
net = network.WideResNet(28, 10, 0.3, 10)
net=  torch.nn.DataParallel(net)
PATH = f'C:\\Users\\Admin\\Desktop\\Python\\corruption-testing\\experiments\\models\\CIFAR10\\wrn28\\ManualSteps\\combined_training\\wrn28_config13_concurrent_1_run_4.pth'
state_dict = torch.load(PATH)
net.load_state_dict(state_dict["net"], strict=True)

<All keys matched successfully>

In [17]:
BATCH_SIZE = 1

transform = transforms.Compose([transforms.ToTensor()])
testset = datasets.CIFAR10(root="./experiments/data",
                           train=False,
                           transform=transform,
                           download=True)

testloader = torch.utils.data.DataLoader(testset,
                                         batch_size=BATCH_SIZE,
                                         shuffle=True,
                                         num_workers=2)

truncated_testset, _ = torch.utils.data.random_split(testset, 
                                                     [50, 9950],
                                                     generator=torch.Generator().manual_seed(42))

truncated_testloader = torch.utils.data.DataLoader(truncated_testset,
                                                   batch_size=BATCH_SIZE,
                                                   shuffle=True,
                                                   num_workers=2)

Files already downloaded and verified


In [18]:
net.eval()

DataParallel(
  (module): WideResNet(
    (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (layer1): Sequential(
      (0): WideBasic(
        (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv1): Conv2d(16, 160, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (dropout): Dropout(p=0.3, inplace=False)
        (bn2): BatchNorm2d(160, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(160, 160, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (shortcut): Sequential(
          (0): Conv2d(16, 160, kernel_size=(1, 1), stride=(1, 1))
        )
      )
      (1): WideBasic(
        (bn1): BatchNorm2d(160, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv1): Conv2d(160, 160, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (dropout): Dropout(p=0.3, inplace=False)
        (bn2): BatchNorm2d(160, eps=1e-05, momentum=0.1, af

In [19]:
def pgd_with_early_stopping(model, inputs, labels, eps, number_of_iters, epsilon_iters, norm):

  label_flipped = False
  
  outputs = model(inputs)
  _, clean_predicted = torch.max(outputs.data, 1)
  misclassified_idx = (clean_predicted!=labels).nonzero()
  total_labels = clean_predicted if clean_predicted!=labels else labels

  # if clean_predicted!=labels:
  #   count+=1

  for i in range(number_of_iters):

    adv_inputs = projected_gradient_descent(model,
                                            inputs,
                                            eps=eps,
                                            eps_iter=epsilon_iters,
                                            nb_iter=1,
                                            y = labels,
                                            norm=norm,
                                            rand_init=False,
                                            sanity_checks=False)
    

    outputs = model(adv_inputs)

    _, predicted = torch.max(outputs.data, 1)

    label_flipped = bool(predicted!=total_labels)
    
    if label_flipped:
      if clean_predicted!=labels:
        print(f"Iterations for successful attack on misclassified input: {i+1}")
        break
      print(f"Iterations for successful attack: {i+1}")
      break
    inputs = adv_inputs.clone()
  return adv_inputs

In [20]:
torch.cuda.empty_cache()

In [21]:
epsilon = 0.1
eps_iter = 0.0004
nb_list = [1, 2, 3, 5, 7, 10, 15, 20, 25, 30, 40, 50, 60, 70, 80, 90, 100]
norm = np.inf

In [22]:
def adv_distance_calculation(testloader, model, nb):  
  distance_list_0, image_idx_0 = [], []  
  distance_list_1, image_idx_1 = [], []  
  distance_list_2, image_idx_2 = [], []
  
  correct, total = 0, 0
  for i, data in enumerate(testloader):
    inputs, labels = data
    inputs, labels = inputs.to(device), labels.to(device)

    
    outputs = model(inputs)
    _, predicted = torch.max(outputs.data, 1)

    adv_inputs = pgd_with_early_stopping(model, inputs, labels, epsilon, nb, eps_iter, norm)

    distance = torch.norm((inputs-adv_inputs), p=1)
    distance_list_0.append(distance)
    image_idx_0.append(i)

    if (predicted==labels):

      distance = torch.norm((inputs-adv_inputs), p=1)
      distance_list_1.append(distance)
      distance_list_2.append(distance)
      image_idx_1.append(i)
      image_idx_2.append(i)

    else:
      distance_list_1.append(0)
      image_idx_1.append(i)

    adv_outputs = model(adv_inputs)
    _, adv_predicted = torch.max(adv_outputs.data, 1)

    correct += (adv_predicted==labels).sum().item()
    total += labels.size(0)
    adv_acc = correct/total

    if i%100 == 0:
      print(f"Completed: {i}, l1_distance: {distance}, Adversarial accuracy: {adv_acc*100}%")

  return distance_list_0, image_idx_0, distance_list_1, image_idx_1, distance_list_2, image_idx_2, adv_acc

In [ ]:
adversarial_accuracies = []
for nb in nb_list:
  dst0, idx0, dst1, idx1, dst2, idx2, adv_acc = adv_distance_calculation(testloader, net, nb)
  adversarial_accuracies.append(adv_acc*100)

Completed: 0, l1_distance: 1.22882080078125, Adversarial accuracy: 100.0%
Iterations for successful attack: 1
Completed: 100, l1_distance: 1.2288198471069336, Adversarial accuracy: 89.10891089108911%
Iterations for successful attack: 1
Completed: 200, l1_distance: 1.2288156747817993, Adversarial accuracy: 92.03980099502488%
Completed: 300, l1_distance: 1.2288203239440918, Adversarial accuracy: 92.02657807308971%
Iterations for successful attack: 1
Completed: 400, l1_distance: 1.2288193702697754, Adversarial accuracy: 92.26932668329178%
Iterations for successful attack: 1
Completed: 500, l1_distance: 1.2288188934326172, Adversarial accuracy: 92.61477045908184%
Iterations for successful attack: 1
Completed: 600, l1_distance: 1.2288174629211426, Adversarial accuracy: 93.5108153078203%
Completed: 700, l1_distance: 1.22882080078125, Adversarial accuracy: 94.00855920114122%
Iterations for successful attack: 1
Iterations for successful attack: 1
Completed: 800, l1_distance: 1.22882080078125, 

Completed: 6200, l1_distance: 1.2288203239440918, Adversarial accuracy: 94.0815997419771%
Iterations for successful attack: 1
Completed: 6300, l1_distance: 1.22882080078125, Adversarial accuracy: 94.04856372004444%
Iterations for successful attack on misclassified input: 1
Completed: 6400, l1_distance: 1.2288161516189575, Adversarial accuracy: 94.03218247148884%
Iterations for successful attack: 1
Iterations for successful attack: 1
Completed: 6500, l1_distance: 1.228819489479065, Adversarial accuracy: 94.01630518381788%
Completed: 6600, l1_distance: 1.2288203239440918, Adversarial accuracy: 94.03120739281927%
Completed: 6700, l1_distance: 1.228819489479065, Adversarial accuracy: 94.09043426354276%
Completed: 6800, l1_distance: 1.2288181781768799, Adversarial accuracy: 94.16262314365534%
Completed: 6900, l1_distance: 1.2288193702697754, Adversarial accuracy: 94.20373858861034%
Iterations for successful attack: 1
Completed: 7000, l1_distance: 1.2288206815719604, Adversarial accuracy: 94

Completed: 2000, l1_distance: 2.376835346221924, Adversarial accuracy: 92.65367316341829%
Iterations for successful attack: 2
Iterations for successful attack: 1
Iterations for successful attack: 2
Iterations for successful attack: 1
Iterations for successful attack: 1
Completed: 2100, l1_distance: 2.3456387519836426, Adversarial accuracy: 92.43217515468825%
Iterations for successful attack: 1
Iterations for successful attack: 2
Iterations for successful attack: 2
Iterations for successful attack: 1
Completed: 2200, l1_distance: 2.3648390769958496, Adversarial accuracy: 92.50340754202635%
Completed: 2300, l1_distance: 2.376840114593506, Adversarial accuracy: 92.69882659713168%
Iterations for successful attack: 2
Iterations for successful attack: 1
Completed: 2400, l1_distance: 2.353633165359497, Adversarial accuracy: 92.79466888796335%
Iterations for successful attack: 1
Iterations for successful attack: 2
Completed: 2500, l1_distance: 2.373629331588745, Adversarial accuracy: 92.882846

Iterations for successful attack: 1
Iterations for successful attack: 1
Completed: 6900, l1_distance: 2.3832333087921143, Adversarial accuracy: 92.87059846399073%
Iterations for successful attack: 1
Iterations for successful attack: 2
Iterations for successful attack: 2
Iterations for successful attack: 2
Iterations for successful attack: 2
Completed: 7000, l1_distance: 2.391240358352661, Adversarial accuracy: 92.81531209827168%
Iterations for successful attack: 2
Iterations for successful attack: 1
Completed: 7100, l1_distance: 2.3840320110321045, Adversarial accuracy: 92.81791297000423%
Iterations for successful attack: 2
Completed: 7200, l1_distance: 2.364840030670166, Adversarial accuracy: 92.83432856547702%
Iterations for successful attack: 2
Iterations for successful attack: 1
Iterations for successful attack: 2
Completed: 7300, l1_distance: 2.385636806488037, Adversarial accuracy: 92.82290097246953%
Completed: 7400, l1_distance: 2.3728394508361816, Adversarial accuracy: 92.81178

Iterations for successful attack: 1
Iterations for successful attack: 2
Iterations for successful attack: 1
Iterations for successful attack: 2
Iterations for successful attack: 1
Completed: 1300, l1_distance: 3.532858371734619, Adversarial accuracy: 92.31360491929284%
Iterations for successful attack: 3
Iterations for successful attack: 3
Completed: 1400, l1_distance: 3.5464558601379395, Adversarial accuracy: 92.07708779443254%
Iterations for successful attack: 2
Iterations for successful attack: 2
Iterations for successful attack: 2
Iterations for successful attack: 1
Completed: 1500, l1_distance: 3.5496602058410645, Adversarial accuracy: 92.07195203197868%
Iterations for successful attack: 1
Iterations for successful attack: 3
Iterations for successful attack: 3
Completed: 1600, l1_distance: 3.5727977752685547, Adversarial accuracy: 92.0049968769519%
Iterations for successful attack: 1
Iterations for successful attack: 3
Iterations for successful attack: 2
Iterations for successful 

...

Iterations for successful attack: 32
Iterations for successful attack: 12
Iterations for successful attack: 35
Iterations for successful attack: 15
Iterations for successful attack: 31
Iterations for successful attack: 15
Iterations for successful attack: 35
Iterations for successful attack: 24
Iterations for successful attack: 31
Iterations for successful attack: 8
Iterations for successful attack: 20
Iterations for successful attack: 10
Iterations for successful attack: 14
Iterations for successful attack: 38
Iterations for successful attack: 27
Iterations for successful attack: 34
Iterations for successful attack: 24
Iterations for successful attack: 14
Iterations for successful attack: 60
Iterations for successful attack: 26
Iterations for successful attack: 21
Iterations for successful attack: 13
Iterations for successful attack: 1
Iterations for successful attack: 2
Iterations for successful attack: 2
Iterations for successful attack: 5
Iterations for successful attack: 2
Iterati

Iterations for successful attack: 31
Iterations for successful attack: 9
Iterations for successful attack: 16
Iterations for successful attack: 40
Iterations for successful attack: 50
Iterations for successful attack: 29
Iterations for successful attack: 21
Iterations for successful attack: 15
Iterations for successful attack: 22
Iterations for successful attack: 4
Iterations for successful attack: 23
Iterations for successful attack: 26
Iterations for successful attack: 26
Iterations for successful attack: 37
Iterations for successful attack: 10
Completed: 1300, l1_distance: 11.460968017578125, Adversarial accuracy: 6.994619523443505%
Iterations for successful attack: 19
Iterations for successful attack: 28
Iterations for successful attack: 9
Iterations for successful attack: 21
Iterations for successful attack: 35
Iterations for successful attack: 21
Iterations for successful attack: 27
Iterations for successful attack: 13
Iterations for successful attack: 18
Iterations for successfu

Iterations for successful attack: 10
Iterations for successful attack: 16
Iterations for successful attack: 26
Iterations for successful attack: 21
Iterations for successful attack: 15
Iterations for successful attack: 25
Iterations for successful attack: 35
Iterations for successful attack: 48
Iterations for successful attack: 20
Iterations for successful attack: 21
Iterations for successful attack: 10
Iterations for successful attack: 21
Iterations for successful attack: 3
Iterations for successful attack: 22
Iterations for successful attack: 18
Iterations for successful attack: 57
Iterations for successful attack: 42
Iterations for successful attack: 21
Iterations for successful attack: 23
Iterations for successful attack: 27
Iterations for successful attack: 24
Iterations for successful attack: 20
Iterations for successful attack: 8
Iterations for successful attack: 6
Iterations for successful attack: 52
Iterations for successful attack: 16
Iterations for successful attack: 58
Iter

Iterations for successful attack: 18
Iterations for successful attack: 9
Iterations for successful attack: 20
Iterations for successful attack: 14
Iterations for successful attack: 17
Iterations for successful attack: 32
Iterations for successful attack: 32
Iterations for successful attack: 9
Iterations for successful attack: 5
Iterations for successful attack: 32
Iterations for successful attack: 5
Iterations for successful attack: 49
Iterations for successful attack: 26
Iterations for successful attack: 20
Iterations for successful attack: 10
Iterations for successful attack: 2
Completed: 1800, l1_distance: 2.369633674621582, Adversarial accuracy: 7.329261521377013%
Iterations for successful attack: 30
Iterations for successful attack: 31
Iterations for successful attack: 26
Iterations for successful attack: 10
Iterations for successful attack: 44
Iterations for successful attack: 19
Iterations for successful attack: 6
Iterations for successful attack: 2
Iterations for successful att

Iterations for successful attack: 26
Iterations for successful attack: 1
Iterations for successful attack: 5
Iterations for successful attack: 34
Iterations for successful attack: 43
Iterations for successful attack: 30
Iterations for successful attack: 15
Iterations for successful attack: 57
Iterations for successful attack: 12
Iterations for successful attack: 24
Iterations for successful attack: 49
Iterations for successful attack: 29
Iterations for successful attack: 3
Iterations for successful attack: 11
Iterations for successful attack: 7
Iterations for successful attack: 41
Iterations for successful attack: 57
Iterations for successful attack: 24
Iterations for successful attack: 19
Iterations for successful attack: 1
Iterations for successful attack: 17
Iterations for successful attack: 22
Iterations for successful attack: 36
Iterations for successful attack: 25
Iterations for successful attack: 24
Iterations for successful attack: 13
Iterations for successful attack: 41
Iterat

In [ ]:
def conv(input_metric):
  input_metric_tensor = torch.tensor(input_metric)
  input_metric_np = input_metric_tensor.cpu().numpy()
  return input_metric_np

In [ ]:
import csv
!pip install plotly
import plotly.graph_objs as go
import plotly.offline as pyo

distance_list_0_np = conv(dst0)
distance_list_1_np = conv(dst1)
distance_list_2_np = conv(dst2)

#with open('\\notebooks', "w", newline="") as csv_file:
#  writer = csv.writer(csv_file)
#  writer.writerows([distance_list_0_np, distance_list_1_np, distance_list_2_np])


# Create trace objects for each line plot
marker_size = 3

trace_0 = go.Scatter(x=idx0, y=distance_list_0_np, mode='markers', name='L1 distances (including misclassified inputs)', marker=dict(color='red', size = marker_size))
trace_1 = go.Scatter(x=idx1, y=distance_list_1_np, mode='markers', name='L1 distances (distance=0 for misclassified inputs)', marker=dict(color='green', size = marker_size))
trace_2 = go.Scatter(x=idx2, y=distance_list_2_np, mode='markers', name='L1 distances (excluding for misclassified inputs)', marker=dict(color='blue', size = marker_size))

# Create a data list containing the trace objects
data = [trace_0, trace_1, trace_2]

# Define the layout for the plot
layout = go.Layout(title='L1 Distances for all Inputs and Misclassified Inputs', xaxis=dict(title='Image index'), yaxis=dict(title='L1 distance'))

# Create a figure object
fig = go.Figure(data=data, layout=layout)

# Display the plot in the notebook
pyo.iplot(fig)


In [ ]:
import seaborn as sns

linewidth = 5

sns.kdeplot(distance_list_2_np, label = 'L1 distances (excluding for misclassified inputs)', linewidth=linewidth)
sns.kdeplot(distance_list_1_np, label = "L1 distances (distance=0 for misclassified inputs)", linewidth=linewidth)
sns.kdeplot(distance_list_0_np, label = "L1 distances (including for misclassified inputs)")
sns.despine()
plt.xlabel("L1 distance")
plt.ylabel("Density")
plt.title("Distribution Curve of L1 Distance")
plt.legend()

In [ ]:
correct, total = 0, 0
for i, data in enumerate(testloader):
  inputs, labels = data
  inputs, labels = inputs.to(device), labels.to(device)

  outputs = net(inputs)
  _, predicted = torch.max(outputs.data, 1)

  total += labels.size(0)
  correct += (predicted==labels).sum().item()

acc = correct/total

In [ ]:
print(f"Adversarial Accuracy: {adv_acc*100}%")
print(f"Clean Accuracy: {acc*100}%")
print(f"Average Distance_0: {np.mean(distance_list_0_np)}")
print(f"Average Distance_1: {np.mean(distance_list_1_np)}")
print(f"Average Distance_2: {np.mean(distance_list_2_np)}")

In [ ]:
# adversarial_accuracies = [accur*100 for accur in adversarial_accuracies]
list = [0]
acc_p = acc*100
accuracies = [acc_p]
list.extend(nb_list)
accuracies.extend(adversarial_accuracies)

plt.figure
plt.plot(list, accuracies)
plt.xlabel("Number of Iterations")
plt.ylabel("Adversarial Acccuracy")
plt.savefig(f'C:\\Users\\Admin\\Desktop\\Python\\corruption-testing\\notebooks\\iters-adv-acc.pdf', dpi=300)
plt.savefig(f'C:\\Users\\Admin\\Desktop\\Python\\corruption-testing\\notebooks\\iters-adv-acc.png', dpi=300)